# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
%%time
data = get_date_list(data)
data = read_data(data)

Date List Created
Record num : 152720
Wall time: 14.3 s


In [3]:
%%time
data['df']['tick']      = (data["df"]['Ask'] + data["df"]['Bid'])/2
data['df']['DateTime_frmt']   = pd.to_datetime(data["df"]['DateTime'], format="%Y%m%d %H:%M:%S")
data['df_ohlc'] = data['df'][['DateTime_frmt', 'tick']]

Wall time: 35.3 ms


In [4]:
data['df_ohlc'] = data['df'][['DateTime_frmt', 'tick']]
data['candle_size'] = '15T'

In [5]:
%%time
data = get_ohlc(data)

Wall time: 32.9 ms


In [6]:
# Hammer

data['df_ohlc']['cdl_hammer'] = talib.CDLHAMMER(data['df_ohlc']['open'], data['df_ohlc']['high'], data['df_ohlc']['low'], data['df_ohlc']['close'])

# Engulfing candle

data['df_ohlc']['cdl_engulfing'] = talib.CDLENGULFING(data['df_ohlc']['open'], data['df_ohlc']['high'], data['df_ohlc']['low'], data['df_ohlc']['close'])

### Merge with DF to visualize Data

In [7]:
data = merge_ohlc_data(data)

100%|██████████████████████████████████████████| 96/96 [00:01<00:00, 66.79it/s]


In [8]:
data['df']['cdl_hammer'] = data['df']['cdl_hammer'].replace({0:np.nan})
data['df']['cdl_hammer'] = np.where(data['df']['cdl_hammer'] == 100, data['df']['tick'], data['df']['cdl_hammer'])

In [9]:
data['df']['cdl_engulfing_up'] = data['df']['cdl_engulfing'][data['df']['cdl_engulfing'] == 100]
data['df']['cdl_engulfing_down'] = data['df']['cdl_engulfing'][data['df']['cdl_engulfing'] == -100]

data['df']['cdl_engulfing_up'] = data['df']['cdl_engulfing_up'].replace({0:np.nan})
data['df']['cdl_engulfing_up'] = np.where(data['df']['cdl_engulfing_up'] == 100, data['df']['tick'], data['df']['cdl_engulfing_up'])

data['df']['cdl_engulfing_down'] = data['df']['cdl_engulfing_down'].replace({0:np.nan})
data['df']['cdl_engulfing_down'] = np.where(data['df']['cdl_engulfing_down'] == -100, data['df']['tick'], data['df']['cdl_engulfing_down'])

del data['df']['cdl_engulfing'] 

### Plot

In [11]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
data['plot_start']  = '2022-07-19'
data['plot_stop']   = '2022-07-20'
data['things_to_plot'] = ['tick', 'positions']

plot_graph(data)

sys.exit()

data['identifier_type'] = 'cdl_hammer'
data['identifier_val'] = 100
data['test_type'] = 'up'

data = calculate_total_profit(data)

data['identifier_type'] = 'cdl_engulfing'
data['identifier_val'] = 100
data['test_type'] = 'up'
data = calculate_total_profit(data)

data['identifier_type'] = 'cdl_engulfing'
data['identifier_val'] = -100
data['test_type'] = 'down'

data = calculate_total_profit(data)